## Final Project Submission

Please fill out:
* Student name: Esther Terry Munene
* Student pace: full time
* Scheduled project review date/time: march 22nd 2024
* Instructor name: MaryAnne Mwikali
* Blog post URL: N/A


# Microsoft's New Movie Studios

## 1.Introduction
 
 ## Overview of the project
 

In [7]:
# Your code here - remember to use markdown cells for comments as well!
#importing libraries
import pandas as pd
import sqlite3
import csv
import numpy as np
from datetime import datetime
import gzip 


In [8]:
# importing data visualisation tools 
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


## Loading the data

Now is time to start working with the  movie datasets stored in the 'ZippedData' folder. We will simply run the cell below to import the tables from the database as DataFrames.
movies_budget = pd.read_csv('zippedData/tn.movie_budgets.csv')
movies_gross = pd.read_csv('zippedData/bom.movie_gross.csv')
tmd = pd.read_csv('zippedData/tmdb.movies.csv',index_col=0)
reviews = pd.read_csv('zippedData/rt.reviews.tsv', delimiter='\t',encoding='unicode_escape')
conn = sqlite3.connect('zippedData/im.db')
movie_info = pd.read_csv('zippedData/rt.movie_info.tsv', delimiter ='\t', encoding='unicode_escape' )




# Exploring Datasets
## Number Dataset
The Numbers dataset gives us information on the budget and returns.



In [24]:
movie_budget=pd.read_csv("zippedData/tn.movie_budgets.csv.gz")
movie_budget
#Cast to numeric data type
movie_budget['production_budget'] = movie_budget['production_budget'].replace('[^.0-9]', '', regex=True).astype('float32')
movie_budget['domestic_gross'] = movie_budget['domestic_gross'].replace('[^.0-9]', '', regex=True).astype('float32')
movie_budget['worldwide_gross'] = movie_budget['worldwide_gross'].replace('[^.0-9]', '',regex = True).astype('float64')
movie_budget.head(5)


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000.0,760507648.0,2.776345e+09
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063872.0,1.045664e+09
2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762352.0,1.497624e+08
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005856.0,1.403014e+09
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181376.0,1.316722e+09


In [25]:

movie_budget.shape

(5782, 6)

In [26]:
movie_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5782 non-null   int64  
 1   release_date       5782 non-null   object 
 2   movie              5782 non-null   object 
 3   production_budget  5782 non-null   float32
 4   domestic_gross     5782 non-null   float32
 5   worldwide_gross    5782 non-null   float64
dtypes: float32(2), float64(1), int64(1), object(2)
memory usage: 226.0+ KB


In [18]:
movie_budget.isna().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [27]:
movie_budget.duplicated().sum()

0

In [28]:
#getting the international gross
movie_budget['international_gross'] = movie_budget['worldwide_gross'] - movie_budget['domestic_gross']
cols = list(movie_budget.columns)

movie_budget.head(10)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,international_gross
0,1,"Dec 18, 2009",Avatar,425000000.0,760507648.0,2.776345e+09,2.015838e+09
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063872.0,1.045664e+09,8.046000e+08
2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762352.0,1.497624e+08,1.070000e+08
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005856.0,1.403014e+09,9.440081e+08
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181376.0,1.316722e+09,6.965404e+08
5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000.0,936662208.0,2.053311e+09,1.116649e+09
6,7,"Apr 27, 2018",Avengers: Infinity War,300000000.0,678815488.0,2.048134e+09,1.369319e+09
7,8,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,300000000.0,309420416.0,9.634204e+08,6.540000e+08
8,9,"Nov 17, 2017",Justice League,300000000.0,229024288.0,6.559452e+08,4.269209e+08
9,10,"Nov 6, 2015",Spectre,300000000.0,200074176.0,8.796209e+08,6.795467e+08


In [30]:
#converting object to dateTime
movie_budget['release_date'] = pd.to_datetime(movie_budget['release_date'], errors='coerce')
movie_budget['year'] = movie_budget['release_date'].dt.strftime('%Y')
movie_budget.head(10)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,international_gross,year
0,1,2009-12-18,Avatar,425000000.0,760507648.0,2.776345e+09,2.015838e+09,2009
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063872.0,1.045664e+09,8.046000e+08,2011
2,3,2019-06-07,Dark Phoenix,350000000.0,42762352.0,1.497624e+08,1.070000e+08,2019
3,4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005856.0,1.403014e+09,9.440081e+08,2015
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181376.0,1.316722e+09,6.965404e+08,2017
5,6,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000.0,936662208.0,2.053311e+09,1.116649e+09,2015
6,7,2018-04-27,Avengers: Infinity War,300000000.0,678815488.0,2.048134e+09,1.369319e+09,2018
7,8,2007-05-24,Pirates of the Caribbean: At Worldâs End,300000000.0,309420416.0,9.634204e+08,6.540000e+08,2007
8,9,2017-11-17,Justice League,300000000.0,229024288.0,6.559452e+08,4.269209e+08,2017
9,10,2015-11-06,Spectre,300000000.0,200074176.0,8.796209e+08,6.795467e+08,2015


In [31]:
movie_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   5782 non-null   int64         
 1   release_date         5782 non-null   datetime64[ns]
 2   movie                5782 non-null   object        
 3   production_budget    5782 non-null   float32       
 4   domestic_gross       5782 non-null   float32       
 5   worldwide_gross      5782 non-null   float64       
 6   international_gross  5782 non-null   float64       
 7   year                 5782 non-null   object        
dtypes: datetime64[ns](1), float32(2), float64(2), int64(1), object(2)
memory usage: 316.3+ KB


Data cleaning
Filtering the data
Here we will drop the rows without the values for the domestic, international and worldwide gross.



In [33]:
movie_budget.sort_values('worldwide_gross')

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,international_gross,year
5037,38,2019-04-23,Living Dark: The Story of Ted the Caver,1750000.0,0.0,0.000000e+00,0.000000e+00,2019
3975,76,2015-05-15,Pound of Flesh,7500000.0,0.0,0.000000e+00,0.000000e+00,2015
4627,28,2011-06-28,2:13,3500000.0,0.0,0.000000e+00,0.000000e+00,2011
4628,29,2013-01-29,"Batman: The Dark Knight Returns, Part 2",3500000.0,0.0,0.000000e+00,0.000000e+00,2013
3947,48,2019-06-21,Burn Your Maps,8000000.0,0.0,0.000000e+00,0.000000e+00,2019
...,...,...,...,...,...,...,...,...
33,34,2015-06-12,Jurassic World,215000000.0,652270656.0,1.648855e+09,9.965842e+08,2015
6,7,2018-04-27,Avengers: Infinity War,300000000.0,678815488.0,2.048134e+09,1.369319e+09,2018
5,6,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000.0,936662208.0,2.053311e+09,1.116649e+09,2015
42,43,1997-12-19,Titanic,200000000.0,659363968.0,2.208208e+09,1.548844e+09,1997


In [34]:
#drop all all rows without the gross values
movie_budget_drop = movie_budget[(movie_budget['domestic_gross'] == 0) & (movie_budget['worldwide_gross'] == 0)]

In [35]:
movie_budget.drop(movie_budget_drop.index, axis=0, inplace=True)
movie_budget.head(5)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,international_gross,year
0,1,2009-12-18,Avatar,425000000.0,760507648.0,2.776345e+09,2.015838e+09,2009
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063872.0,1.045664e+09,8.046000e+08,2011
2,3,2019-06-07,Dark Phoenix,350000000.0,42762352.0,1.497624e+08,1.070000e+08,2019
3,4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005856.0,1.403014e+09,9.440081e+08,2015
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181376.0,1.316722e+09,6.965404e+08,2017


let's filter the years to start from 2011 in order to have relevant budgeting information.


In [37]:
#convert from object to string
movie_budget['year'] = movie_budget['year'].astype(int)

In [38]:
#drop rows which are not between the years 2015 and 2022
drop_=  movie_budget[(movie_budget['year'] <= 2014)]
movie_budget.drop(drop_.index, axis=0, inplace=True)
movie_budget.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 797 entries, 2 to 5748
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   797 non-null    int64         
 1   release_date         797 non-null    datetime64[ns]
 2   movie                797 non-null    object        
 3   production_budget    797 non-null    float32       
 4   domestic_gross       797 non-null    float32       
 5   worldwide_gross      797 non-null    float64       
 6   international_gross  797 non-null    float64       
 7   year                 797 non-null    int32         
dtypes: datetime64[ns](1), float32(2), float64(2), int32(1), int64(1), object(1)
memory usage: 46.7+ KB


In [39]:
movie_budget['year'].value_counts()

2015    242
2016    204
2017    162
2018    140
2019     49
Name: year, dtype: int64

In [40]:
movie_budget.describe()

,id,production_budget,domestic_gross,worldwide_gross,international_gross,year
count,797.000000,797.0,797.0,7.970000e+02,7.970000e+02,797.000000
mean,50.634881,44454288.0,57354036.0,1.477667e+08,9.041263e+07,2016.435383
std,29.339221,57002492.0,98196584.0,2.607258e+08,1.710463e+08,1.255101
min,1.000000,25000.0,0.0,2.600000e+01,-1.000000e+00,2015.000000
25%,25.000000,8000000.0,3000342.0,8.200000e+06,2.053330e+06,2015.000000
50%,50.000000,22000000.0,24431472.0,4.517374e+07,1.928622e+07,2016.000000
75%,78.000000,55000000.0,63859436.0,1.594876e+08,9.237100e+07,2017.000000
max,100.000000,350000000.0,936662208.0,2.053311e+09,1.369319e+09,2019.000000


 Finding the percentage profits of the movies based on the production budget and the world wide gross.

In [41]:
movie_budget['profit/loss'] = (movie_budget['worldwide_gross'] - movie_budget['production_budget'])
movie_budget['profit/loss'] = movie_budget['profit/loss'].astype(int)      
movie_budget['profit/loss'].tail()

5701    -66201
5719    -41626
5737    114822
5744    -23453
5748    464792
Name: profit/loss, dtype: int32

In [42]:
movie_budget.shape

(797, 9)

In [43]:
movie_budget

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,international_gross,year,profit/loss
2,3,2019-06-07,Dark Phoenix,350000000.0,42762352.0,1.497624e+08,1.070000e+08,2019,-200237650
3,4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005856.0,1.403014e+09,9.440081e+08,2015,1072413963
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181376.0,1.316722e+09,6.965404e+08,2017,999721747
5,6,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000.0,936662208.0,2.053311e+09,1.116649e+09,2015,1747311220
6,7,2018-04-27,Avengers: Infinity War,300000000.0,678815488.0,2.048134e+09,1.369319e+09,2018,1748134200
...,...,...,...,...,...,...,...,...,...
5701,2,2015-07-17,Dawn of the Crescent Moon,75000.0,8799.0,8.799000e+03,0.000000e+00,2015,-66201
5719,20,2015-07-31,Counting,50000.0,8374.0,8.374000e+03,0.000000e+00,2015,-41626
5737,38,2016-03-18,Krisha,30000.0,144822.0,1.448220e+05,0.000000e+00,2016,114822
5744,45,2017-01-27,Emily,27000.0,3547.0,3.547000e+03,0.000000e+00,2017,-23453



### Box Office Movie Dataset
This dataset has domestic_gross and foreign_gross information.

In [44]:
# loading the second dataset
movie_gross=pd.read_csv("zippedData/bom.movie_gross.csv.gz")
movie_gross.tail(10)

,title,studio,domestic_gross,foreign_gross,year
3377,Let Yourself Go,Men.,12200.0,NaN,2018
3378,Hannah (2018),PDF,11700.0,NaN,2018
3379,Souvenir,Strand,11400.0,NaN,2018
3380,Furious (Legend of Kolovrat),CARUSEL,10000.0,NaN,2018
3381,Beauty and the Dogs,Osci.,8900.0,NaN,2018
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018
3386,An Actor Prepares,Grav.,1700.0,NaN,2018


### Understanding the Bom Movie Dataset


In [45]:
movie_gross.shape

(3387, 5)

In [46]:
movie_gross['year'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

In [47]:
movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [48]:
movie_gross.isna().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

In [49]:
#drop the rows without gross values
movie_gross.dropna(subset=['foreign_gross','domestic_gross'], inplace=True)
movie_gross.isna().sum()

title             0
studio            2
domestic_gross    0
foreign_gross     0
year              0
dtype: int64

### Movie DB Dataset

In [51]:
tmdb_movie= pd.read_csv('zippedData/tmdb.movies.csv.gz', index_col=0)
tmdb_movie.head(5)

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
